## Notebook destinado ao tratamento da base de dados 'Forest Fires in Brazil'

In [89]:
import pandas as pd
import numpy as np

#### Ao tentar importar a base diretamente, você receberá um erro, isso se dá pois a base de dados possui valores como 'ç' e letras acentuadas, o decodificador padrão do pandas não interpreta esses valores, para isso usaremos o decodificador 'latin1' como parâmetro na função 'read_csv()'. Além disso, os valores maiores que 999 foram computados com  um ponto '.' dividindo a casa dos milhares, para amenizar o problema, vamos usar o parâmetro decimals=',' que nos ajudará no tratamento dos dados

In [47]:
df_incendios = pd.read_csv('amazon.csv', encoding='latin1', decimal=',')
df_incendios.head()

,year,state,month,number,date
0,1998,Acre,Janeiro,0,1998-01-01
1,1999,Acre,Janeiro,0,1999-01-01
2,2000,Acre,Janeiro,0,2000-01-01
3,2001,Acre,Janeiro,0,2001-01-01
4,2002,Acre,Janeiro,0,2002-01-01


A base foi importada, vamos checar as informações:

In [48]:
df_incendios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6454 entries, 0 to 6453
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   year    6454 non-null   int64 
 1   state   6454 non-null   object
 2   month   6454 non-null   object
 3   number  6454 non-null   object
 4   date    6454 non-null   object
dtypes: int64(1), object(4)
memory usage: 252.2+ KB


Olhando os valores da coluna 'state' podemos observar alguns problemas:

In [49]:
df_incendios['state'].unique()

array(['Acre', 'Alagoas', 'Amapa', 'Amazonas', 'Bahia', 'Ceara',
       'Distrito Federal', 'Espirito Santo', 'Goias', 'Maranhao',
       'Mato Grosso', 'Minas Gerais', 'Pará', 'Paraiba', 'Pernambuco',
       'Piau', 'Rio', 'Rondonia', 'Roraima', 'Santa Catarina',
       'Sao Paulo', 'Sergipe', 'Tocantins'], dtype=object)

Podemos observar que existem problemas na base de dados:
- Um estado chamado 'Rio'
- Faltam os estados do Paraná e do Mato Grosso do Sul
- O estado do Piauí está escrito apenas como 'Piau'

Vamos visualizar a quantidade de registros por estado, para facilitar nosso tratamento dos dados

In [51]:
df_incendios['state'].value_counts()

state
Rio                 717
Paraiba             478
Mato Grosso         478
Alagoas             240
Acre                239
Sergipe             239
Sao Paulo           239
Santa Catarina      239
Roraima             239
Rondonia            239
Piau                239
Pernambuco          239
Minas Gerais        239
Pará                239
Maranhao            239
Goias               239
Espirito Santo      239
Distrito Federal    239
Ceara               239
Bahia               239
Amazonas            239
Amapa               239
Tocantins           239
Name: count, dtype: int64

Percebe-se que:
- O estado 'Rio' possui 3 estados combinados (Rio de Janeiro, Rio Grande do Norte e Rio Grande do Sul)
- O estado 'Mato Grosso' possui outro estado combinado a ele (Mato Grosso do Sul)
- O estado 'Paraiba' possui outro estado combinado a ele (Paraná)
- O estado de Alagoas possui um registro a mais do que os demais

### Vamos realizar o tratamento por partes:

#### Tratamento 1 - Alagoas possui 1 registro a mais:

Primeiro, criei um dataframe reduzido apenas para Alagoas, facilitando a análise

In [52]:
df_alagoas = df_incendios[df_incendios['state']=='Alagoas']
print(df_alagoas.head())
print(df_alagoas.tail())

     year    state    month number        date
239  1998  Alagoas  Janeiro      0  1998-01-01
240  1999  Alagoas  Janeiro     58  1999-01-01
241  2000  Alagoas  Janeiro     11  2000-01-01
242  2001  Alagoas  Janeiro      5  2001-01-01
243  2002  Alagoas  Janeiro     12  2002-01-01
     year    state     month number        date
474  2012  Alagoas  Dezembro     59  2012-01-01
475  2013  Alagoas  Dezembro     42  2013-01-01
476  2014  Alagoas  Dezembro     24  2014-01-01
477  2015  Alagoas  Dezembro    136  2015-01-01
478  2016  Alagoas  Dezembro    155  2016-01-01


Com o dataframe reduzido, usei a função 'duplicated' para procurar duplicatas, o parâmetro keep=False marca todos os registros duplicados.

In [41]:
df_alagoas[df_alagoas.duplicated(keep=False)]

,year,state,month,number,date
258,2017,Alagoas,Janeiro,38.0,2017-01-01
259,2017,Alagoas,Janeiro,38.0,2017-01-01


Achamos o valor duplicado de alagoas, vamos remove-lo do DataFrame principal (df_incendios)

In [53]:
df_incendios.drop(index = 259, inplace=True)

# resetando o indice do dataframe
df_incendios.reset_index(drop=True, inplace=True)

In [54]:
# verificando novamente a contagem de registros por estado
df_incendios['state'].value_counts()

state
Rio                 717
Mato Grosso         478
Paraiba             478
Acre                239
Pará                239
Sergipe             239
Sao Paulo           239
Santa Catarina      239
Roraima             239
Rondonia            239
Piau                239
Pernambuco          239
Minas Gerais        239
Alagoas             239
Maranhao            239
Goias               239
Espirito Santo      239
Distrito Federal    239
Ceara               239
Bahia               239
Amazonas            239
Amapa               239
Tocantins           239
Name: count, dtype: int64

---

#### Tratamento 2 - Mato Grosso e Mato Grosso do Sul combinados em um único estado:

Como observado que cada estado possui 239 registros, podemos dividir os 478 registros de 'Mato Grosso' em 2 partes iguais.
Supondo a ordem alfabética da base de dados, a ordem deve ser, primeiro Mato Grosso e depois Mato Grosso do Sul

Utilizando a mesma abordagem de criar um dataframe para nossa análise:

In [55]:
df_mato_grosso = df_incendios[df_incendios['state']=='Mato Grosso']
df_mato_grosso

,year,state,month,number,date
2390,1998,Mato Grosso,Janeiro,0,1998-01-01
2391,1999,Mato Grosso,Janeiro,39,1999-01-01
2392,2000,Mato Grosso,Janeiro,44,2000-01-01
2393,2001,Mato Grosso,Janeiro,44,2001-01-01
2394,2002,Mato Grosso,Janeiro,172,2002-01-01
...,...,...,...,...,...
2863,2012,Mato Grosso,Dezembro,194,2012-01-01
2864,2013,Mato Grosso,Dezembro,195,2013-01-01
2865,2014,Mato Grosso,Dezembro,84,2014-01-01
2866,2015,Mato Grosso,Dezembro,394,2015-01-01


Agora, dividindo o intervalo, temos que:
- As posições para Mato Grosso vão de 2390 a 2628
- As posições para Mato Grosso do Sul vão de 2629 a 2867

Só precisamos alterar os registros do Mato Grosso do Sul diretamente no DataFrame principal 'df_incendios'

In [56]:
df_incendios.loc[2629:2867, 'state'] = 'Mato Grosso do Sul'

In [57]:
# Visualizando a quantidade de registros por estados após a alteração feita
df_incendios['state'].value_counts()

state
Rio                   717
Paraiba               478
Acre                  239
Alagoas               239
Sergipe               239
Sao Paulo             239
Santa Catarina        239
Roraima               239
Rondonia              239
Piau                  239
Pernambuco            239
Pará                  239
Minas Gerais          239
Mato Grosso do Sul    239
Mato Grosso           239
Maranhao              239
Goias                 239
Espirito Santo        239
Distrito Federal      239
Ceara                 239
Bahia                 239
Amazonas              239
Amapa                 239
Tocantins             239
Name: count, dtype: int64

---

#### Tratamento 3 - Estado do Paraná combinado no estado da Paraíba:

 Usando a mesma lógica utilizada para a separação feita anteriormente, vamos separar os registros. Na ordem alfabética, os primeiros registros devem ser da Paraiba e os seguintes devem ser do Parana.

In [58]:
df_paraiba = df_incendios[df_incendios['state']=='Paraiba']
df_paraiba

,year,state,month,number,date
3346,1998,Paraiba,Janeiro,0,1998-01-01
3347,1999,Paraiba,Janeiro,26,1999-01-01
3348,2000,Paraiba,Janeiro,0,2000-01-01
3349,2001,Paraiba,Janeiro,11,2001-01-01
3350,2002,Paraiba,Janeiro,5,2002-01-01
...,...,...,...,...,...
3819,2012,Paraiba,Dezembro,54,2012-01-01
3820,2013,Paraiba,Dezembro,155,2013-01-01
3821,2014,Paraiba,Dezembro,78,2014-01-01
3822,2015,Paraiba,Dezembro,30,2015-01-01


Agora, dividindo o intervalo, temos que:
- As posições para Paraiba vão de 3346 a 2584
- As posições para Parana vão de 3585 a 3823

Só precisamos alterar os registros do Parana diretamente no DataFrame principal 'df_incendios'

In [59]:
df_incendios.loc[3585:3823, 'state'] = 'Parana'

In [60]:
# Visualizando a quantidade de registros por estados após a alteração feita
df_incendios['state'].value_counts()

state
Rio                   717
Acre                  239
Pará                  239
Sergipe               239
Sao Paulo             239
Santa Catarina        239
Roraima               239
Rondonia              239
Piau                  239
Pernambuco            239
Parana                239
Paraiba               239
Minas Gerais          239
Alagoas               239
Mato Grosso do Sul    239
Mato Grosso           239
Maranhao              239
Goias                 239
Espirito Santo        239
Distrito Federal      239
Ceara                 239
Bahia                 239
Amazonas              239
Amapa                 239
Tocantins             239
Name: count, dtype: int64

---

#### Tratamento 4 - Estado 'Rio' possui 3 estados combinados 

Usando a mesma lógica dos tratamentos anteriores, vamos separar o estado 'Rio' em:
1. Rio de Janeiro
2. Rio Grande do Norte
3. Rio Grande do Sul

Nessa ordem, dado o ordenamento alfabético.

In [61]:
df_rio = df_incendios[df_incendios['state']=='Rio']
df_rio

,year,state,month,number,date
4302,1998,Rio,Janeiro,0,1998-01-01
4303,1999,Rio,Janeiro,0,1999-01-01
4304,2000,Rio,Janeiro,0,2000-01-01
4305,2001,Rio,Janeiro,0,2001-01-01
4306,2002,Rio,Janeiro,0,2002-01-01
...,...,...,...,...,...
5014,2012,Rio,Dezembro,38,2012-01-01
5015,2013,Rio,Dezembro,62,2013-01-01
5016,2014,Rio,Dezembro,31,2014-01-01
5017,2015,Rio,Dezembro,42,2015-01-01


Como não iremos reutilizar o nome 'Rio' precisamos renomear as 3 partes do intervalo:

- 'Rio de Janeiro' para o intervalo [4302 : 4540]
- 'Rio Grande do Norte' para o intervalo [4541 : 4779]
- 'Rio Grande do Sul' para o intervalo [4780 : 5018]

Agora podemos alterar diretamente no DataFrame principal 'df_incendios'

In [62]:
df_incendios.loc[4302:4540, 'state'] = 'Rio de Janeiro'
df_incendios.loc[4541:4779, 'state'] = 'Rio Grande do Norte'
df_incendios.loc[4780:5018, 'state'] = 'Rio Grande do Sul'

In [64]:
# Visualizando a quantidade de registros por estados após a alteração feita
df_incendios['state'].value_counts()

state
Acre                   239
Paraiba                239
Sergipe                239
Sao Paulo              239
Santa Catarina         239
Roraima                239
Rondonia               239
Rio Grande do Sul      239
Rio Grande do Norte    239
Rio de Janeiro         239
Piau                   239
Pernambuco             239
Parana                 239
Pará                   239
Alagoas                239
Minas Gerais           239
Mato Grosso do Sul     239
Mato Grosso            239
Maranhao               239
Goias                  239
Espirito Santo         239
Distrito Federal       239
Ceara                  239
Bahia                  239
Amazonas               239
Amapa                  239
Tocantins              239
Name: count, dtype: int64

Agora temos os 26 estados mais Distrito Federal igualmente com 239 registros.

---

#### Agora, precisamos tratar a tabela 'number', os valores acima de 999 foram computados com um ponto '.' quando o pandas tenta interpretá-los, os converte para decimais

Ex: O número 1100 é computado como 1.100, o pandas interpreta-o como 1.1

In [65]:
# Vamos observar algumas linhas
df_incendios.loc[2500:2520]

,year,state,month,number,date
2500,2008,Mato Grosso,Junho,403,2008-01-01
2501,2009,Mato Grosso,Junho,979,2009-01-01
2502,2010,Mato Grosso,Junho,1.025,2010-01-01
2503,2011,Mato Grosso,Junho,974,2011-01-01
2504,2012,Mato Grosso,Junho,1.651,2012-01-01
2505,2013,Mato Grosso,Junho,1.213,2013-01-01
2506,2014,Mato Grosso,Junho,1.795,2014-01-01
2507,2015,Mato Grosso,Junho,1.402,2015-01-01
2508,2016,Mato Grosso,Junho,1.417,2016-01-01
2509,2017,Mato Grosso,Junho,2.041,2017-01-01


Para resolver esse problema vou seguir a seguinte estratégia:

- Criar uma coluna 'multiplicador' que receberá 1000 ou 1
- Para cada valor na coluna 'number' checar se o valor possui '.' Se sim: Coluna 'multiplicador' recebe 1000, se não, recebe 1
- Multiplicar o valor em 'number' pelo valor em 'multiplicador'

In [67]:
# criando a coluna multiplicador e o loop para verificar cada valor em 'number'
df_incendios['Multiplicador'] = [1000 if '.' in valor else 1 for valor in df_incendios['number']]

# atribuindo ao valor em number sua multiplicação pela coluna 'multiplicador'
df_incendios['number'] = df_incendios['number'].astype('float64')*df_incendios['Multiplicador']

# excluindo a coluna 'multiplicador' usada apenas como auxiliar
df_incendios.drop(columns='Multiplicador', inplace = True)

In [69]:
df_incendios.loc[2500:2520]

,year,state,month,number,date
2500,2008,Mato Grosso,Junho,403.0,2008-01-01
2501,2009,Mato Grosso,Junho,979.0,2009-01-01
2502,2010,Mato Grosso,Junho,1025.0,2010-01-01
2503,2011,Mato Grosso,Junho,974.0,2011-01-01
2504,2012,Mato Grosso,Junho,1651.0,2012-01-01
2505,2013,Mato Grosso,Junho,1213.0,2013-01-01
2506,2014,Mato Grosso,Junho,1795.0,2014-01-01
2507,2015,Mato Grosso,Junho,1402.0,2015-01-01
2508,2016,Mato Grosso,Junho,1417.0,2016-01-01
2509,2017,Mato Grosso,Junho,2041.0,2017-01-01


Agora a base de dados está pronta para resolvermos as questões:

---

#### 11) Carregue o dataset e crie uma mensagem ao leitor do notebook falando quantas linhas e colunas possui o DataFrame.

Nossa base já foi carregada e tratada acima, vamos apenas imprimir a mensagem para o leitor

In [72]:
 # mensagem informando ao leitor a quantidade de linhas e colunas com o retorno da função shape()
print(f'\nUSUÁRIO: O conjunto de dados possui {df_incendios.shape[0]} linhas e {df_incendios.shape[1]} colunas')


USUÁRIO: O conjunto de dados possui 6453 linhas e 5 colunas


---

#### 12) Utilize loc e iloc para exibir da linha 100 até a linha 120 da segunda, terceira e quarta coluna. Qual a diferença das duas sintaxes feitas?

In [73]:
# primeiro vamos selecionar as linhas e colunas usando o método loc
(df_incendios.loc[100:120, ['state', 'month', 'number']])

,state,month,number
100,Acre,Junho,3.0
101,Acre,Junho,0.0
102,Acre,Junho,1.0
103,Acre,Junho,1.0
104,Acre,Junho,0.0
105,Acre,Junho,0.0
106,Acre,Junho,5.0
107,Acre,Junho,27.0
108,Acre,Junho,1.0
109,Acre,Junho,4.0


In [74]:
# agora vamos fazer a mesma consulta usando o método iloc:
df_incendios.iloc[100:121, [1,2,3]]

,state,month,number
100,Acre,Junho,3.0
101,Acre,Junho,0.0
102,Acre,Junho,1.0
103,Acre,Junho,1.0
104,Acre,Junho,0.0
105,Acre,Junho,0.0
106,Acre,Junho,5.0
107,Acre,Junho,27.0
108,Acre,Junho,1.0
109,Acre,Junho,4.0


As diferenças entre as sintaxes são poucas, enquanto o loc consegue localizar itens pelo seu nome ou valor, o iloc utiliza o índice dos itens para isso. No loc informamos as colunas pelo seu nome, no iloc informamos o indice de cada coluna.

---

#### 13) Qual o total de incêndios que ocorreram no Brasil no ano de 2010? E se considerarmos somente o estado do Mato Grosso do Sul?

In [75]:
# Fazendo um groupby com condição, podemos armazenar a soma de incendios para o ano de 2010
incendios_2010_total = df_incendios.groupby(df_incendios['year'] == 2010)['number'].sum()
incendios_2010_total

year
False    3399732.0
True      249274.0
Name: number, dtype: float64

In [76]:
# Agora, para formatar a resposta, podemos acessar diretamente o valor para 'True'
print(f"A quantidade de incendios em 2010 foi de: {incendios_2010_total[True]}")

A quantidade de incendios em 2010 foi de: 249274.0


#### E se considerarmos somente o estado do Mato Grosso do Sul?

In [79]:
# Criando uma nova tabela apenas com dados do mato grosso do sul em 2010
df_mato_grosso_sul_2010 = df_incendios[(df_incendios['state'] == 'Mato Grosso do Sul') & (df_incendios['year'] == 2010)]
df_mato_grosso_sul_2010.head()

,year,state,month,number,date
2641,2010,Mato Grosso do Sul,Janeiro,72.0,2010-01-01
2661,2010,Mato Grosso do Sul,Fevereiro,157.0,2010-01-01
2681,2010,Mato Grosso do Sul,Março,149.0,2010-01-01
2701,2010,Mato Grosso do Sul,Abril,136.0,2010-01-01
2721,2010,Mato Grosso do Sul,Maio,105.0,2010-01-01


In [81]:
# Agora podemos somar a coluna number
incendios_ms_2010 = df_mato_grosso_sul_2010['number'].sum()
print(f"O total de incendios no Mato Grosso em 2010 foi de: {int(incendios_ms_2010)}")

O total de incendios no Mato Grosso em 2010 foi de: 5715


---

#### 14) Qual a média de incêndios por mês no estado do Acre? E o desvio padrão?


In [83]:
# Criando uma tabela para o Acre, facilitando a consulta

df_acre = df_incendios[df_incendios['state'] == 'Acre']
df_acre.head()

,year,state,month,number,date
0,1998,Acre,Janeiro,0.0,1998-01-01
1,1999,Acre,Janeiro,0.0,1999-01-01
2,2000,Acre,Janeiro,0.0,2000-01-01
3,2001,Acre,Janeiro,0.0,2001-01-01
4,2002,Acre,Janeiro,0.0,2002-01-01


In [85]:
# Agrupando a quantidade total de incendios por mes
df_media_meses_acre = df_acre.groupby('month')['number'].sum()

# Obtendo a média geral por mes
print(f"Media = {df_media_meses_acre.mean()}")

# Obtendo o desvio padrão por mes
print(f"Desvio padrao = {df_media_meses_acre.std()}")

Media = 6119.916666666667
Desvio padrao = 12472.427805205394


---

#### 15) Qual estado possui a maior média de incêndios por mês? E qual possui a menor?


In [86]:
# Vamos agrupar o soma do numero de incendios por estado e mes
media_por_mes = df_incendios.groupby(['state', 'month'])['number'].sum()
media_por_mes

# Criando um array 'estados' para facilitar a análise
estados = df_incendios['state'].unique()

# Criando um dicionario que recebe como chave o nome do estado e como valor a sua média mensal de incendios, facilitando consultas
dict_estado_media_mensal = {}
for estado in estados:
    dict_estado_media_mensal[estado] = media_por_mes[estado].mean()

# Como nossos dados estãoarmazenados em um dicionario, podemos consultas os maiores e menores valores de forma muito simples
print(f"Estado com maior média mensal de incendios: {max(dict_estado_media_mensal)}")
print(f"Estado com menor média mensal de incendios: {min(dict_estado_media_mensal)}")

Estado com maior média mensal de incendios: Tocantins
Estado com menor média mensal de incendios: Acre


---

#### 16) Crie uma coluna chamada regiao_sudeste e marque com 1 para quando a observação pertencer a um destes estados e 0, caso contrário. Faça o mesmo para a região centro-oeste. Qual a diferença no total de incêndios dessas regiões? Qual região possui mais incêndios? Consulte a internet para entender por qual motivo uma das regiões possui mais incêndio que a outra.

In [87]:
# Para isso, dividi os estados em duas listas referentes às regiões
array_sudeste = ['Espirito Santo', 'Minas Gerais', 'Rio de Janeiro', 'São Paulo']
array_centro_oeste = ['Goias', 'Mato Grosso', 'Mato Grosso do Sul', 'Distrito Federal']

In [90]:
# Usando o método 'where' da lib numpy, podemos fazer uma criação de coluna condicional
df_incendios['regiao_sudeste']=np.where(df_incendios['state'].isin(array_sudeste), 1, 0)
df_incendios['regiao_centro_oeste']=np.where(df_incendios['state'].isin(array_centro_oeste), 1, 0)

In [91]:
# Vamos agora agrupar a soma de incendios para as novas colunas criadas
incendios_sudeste = df_incendios.groupby('regiao_sudeste')['number'].sum()
print(incendios_sudeste)

print("\n\n")

incendios_centro_oeste = df_incendios.groupby('regiao_centro_oeste')['number'].sum()
print(incendios_centro_oeste)

regiao_sudeste
0    3449168.0
1     199838.0
Name: number, dtype: float64



regiao_centro_oeste
0    2686781.0
1     962225.0
Name: number, dtype: float64


Podemos observar que, para cada coluna, o agrupamento retornou a soma para 1(estados da região) e para 0(estados de fora da região)

In [93]:
# Para facilitar a visualização dos resultados, vamos acessar a soma para 1 de cada coluna, obtendo nossa resposta da questão:

print(f"Sudeste: {incendios_sudeste[1]}")
print(f"Centro Oeste: {incendios_centro_oeste[1]}")

Sudeste: 199838.0
Centro Oeste: 962225.0


Podemos observar que a região Centro Oeste tem cerca de 54% mais incêndios que a região Sudeste.

Segundo a WWF Brasil, vários fatores contribuem para a grande quantidade de queimadas no centro-oeste, entre eles estão: Baixa umidade do ar, vegetação seca, altas temperaturas e baixa estiagem, somados à ação de latifundiários que promovem queimadas intencionais com diversas finalidades para o agronegócio.